In [2]:

import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import pyrealsense2 as rs
import datetime as dt
import time
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV , cross_val_score
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier , VotingClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import permutation_importance
from sklearn.cluster import KMeans
import logging
import pickle 
# import rospy
# import moveit_commander
import sys
from math import pi
#  from subprocess import *
import serial
from time import sleep
import random



In [3]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [ ]:
font = cv2.FONT_HERSHEY_COMPLEX
org = (20,100)
fontScale = .5
thickness = 1 
color = (0,150,255)
realsense_ctx = rs.context()
connected_devices = [] # List of serial numbers for present cameras
for i in range(len(realsense_ctx.devices)):
        detected_camera =  realsense_ctx.devices[i].get_info(rs.camera_info.serial_number)
        print(f"{detected_camera}")
        connected_devices.append(detected_camera)
        
device = connected_devices[0] # In this example we are only using one camera    
 #device = connected_devices[1] # In this example we are only using one camera    
print(f"{device}")
pipeline = rs.pipeline()    
config = rs.config()    
background_removed_color = 153 # Grey

config.enable_device(device)
stream_res_x = 1280
stream_res_y = 720
stream_fps = 15
config.enable_stream(rs.stream.depth, stream_res_x, stream_res_y, rs.format.z16, stream_fps)
config.enable_stream(rs.stream.color, stream_res_x, stream_res_y, rs.format.bgr8, stream_fps)
profile = pipeline.start(config)
align_to = rs.stream.color
align = rs.align(align_to)
# ====== Get depth Scale ======
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
# ====== Set clipping distance ======
clipping_distance_in_meters = 2
clipping_distance = clipping_distance_in_meters / depth_scale

#Facial Landmark Indexes to be detected 
LEFT_EYE=153
RIGHT_EYE=380
NOSE=1
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while True:
        start_time = dt.datetime.today().timestamp() # Necessary for FPS calculations

                # Get and align frames
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        if not aligned_depth_frame or not color_frame:
            continue

            # Process images
        # Wait for the next frame from the RealSense camera
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert the depth frame to a numpy array
        depth_image = np.asanyarray(depth_frame.get_data())

        # Convert the color frame to a numpy array
        color_image = np.asanyarray(color_frame.get_data())
        color_image = cv2.flip(color_image, 1) 
        
        # Apply colormap on depth image (for visualization)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            
        image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
                       # Image is no longer writeable
        
        # Recolor Feed
               
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        if results.face_landmarks:
            h, w, _ = image.shape
            for idx in [LEFT_EYE, RIGHT_EYE, NOSE]:
                landmark = results.face_landmarks.landmark[idx]
                x, y = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # Green dots for landmarks
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    
pipeline.stop()
cv2.destroyAllWindows()

819312070630
819312070630


I0000 00:00:1741155749.795684    3769 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741155749.796609   10349 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: NV134


KeyboardInterrupt: 

In [ ]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)+len(results.left_hand_landmarks.landmark)
num_coords

In [ ]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
connected_devices = [] # List of serial numbers for present cameras
for i in range(len(realsense_ctx.devices)):
        detected_camera =  realsense_ctx.devices[i].get_info(rs.camera_info.serial_number)
        print(f"{detected_camera}")
        connected_devices.append(detected_camera)
camera_input = input("Select camera: ")
if camera_input == '0':
        camera_id = connected_devices[0]
elif camera_input == '1':
        camera_id = connected_devices[1]
print(camera_input)

In [ ]:
file_name = f'coordsV2_{camera_id}_1.csv'
file_path = os.path.join('/home/Downloads/Non_San/', file_name)
with open(file_path, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)


In [ ]:
def extract_keypoints(results):
    poseLSTM = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    faceLSTM = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lhLSTM = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rhLSTM = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([poseLSTM, faceLSTM, lhLSTM, rhLSTM])

In [ ]:
#Give_pen,Take_pen,Give_tape,take_tape,Give_box,take_box, up, down, left,right, Grip, Release ,Input_2, Need_assist

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('NonV2_data') 

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Kengo_data') 


Collect data

In [4]:
#New
# Directory for saving images and data
# model_folder = 'Model'
# os.makedirs(model_folder, exist_ok=True)  # Create the "Model" folder if it doesn't exist
class_name = input("Enter the class name: ")


try: 
    
    os.makedirs(os.path.join(DATA_PATH, class_name), exist_ok=True)
    
except:
            pass

font = cv2.FONT_HERSHEY_COMPLEX
org = (20,100)
fontScale = .5
thickness = 1 
color = (0,150,255)
realsense_ctx = rs.context()
connected_devices = [] # List of serial numbers for present cameras
for i in range(len(realsense_ctx.devices)):
        detected_camera =  realsense_ctx.devices[i].get_info(rs.camera_info.serial_number)
        print(f"{detected_camera}")
        connected_devices.append(detected_camera)
camera_input = input("Select camera: ")
if camera_input == '0':
        camera_id = connected_devices[0]
elif camera_input == '1':
        camera_id = connected_devices[1]
print(camera_input)    
device = camera_id # In this example we are only using one camera  


print(camera_id)
# print(camera_id)

LEFT_EYE=153
RIGHT_EYE=380
NOSE=1

pipeline = rs.pipeline()    
config = rs.config()    
background_removed_color = 153 # Grey

config.enable_device(device)
stream_res_x = 640
stream_res_y = 480
stream_fps = 30
config.enable_stream(rs.stream.depth, stream_res_x, stream_res_y, rs.format.z16, stream_fps)
config.enable_stream(rs.stream.color, stream_res_x, stream_res_y, rs.format.bgr8, stream_fps)
profile = pipeline.start(config)
align_to = rs.stream.color
align = rs.align(align_to)
# ====== Get depth Scale ======
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
# ====== Set clipping distance ======
clipping_distance_in_meters = 2
clipping_distance = clipping_distance_in_meters / depth_scale

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    start_time = time.time()  
    while True:
        
        # start_time = dt.datetime.today().timestamp() # Necessary for FPS calculations

        # count down
        current_time = time.time()
        elapsed_time = current_time - start_time
        remaining_time = max(0, 30 - elapsed_time)  # Countdown for 30 seconds

        if remaining_time == 0:
            break

                # Get and align frames
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        if not aligned_depth_frame or not color_frame:
            continue

            # Process images
        # Wait for the next frame from the RealSense camera
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        
        if not depth_frame or not color_frame:
            continue

        # Convert the depth frame to a numpy array
        depth_image = np.asanyarray(depth_frame.get_data())

        # Convert the color frame to a numpy array
        color_image = np.asanyarray(color_frame.get_data())
        color_image = cv2.flip(color_image, 1) 
        
        # Apply colormap on depth image (for visualization)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            
        image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
                       # Image is no longer writeable
        
        # Recolor Feed
               
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        if results.face_landmarks:
            h, w, _ = image.shape
            for idx in [LEFT_EYE, RIGHT_EYE, NOSE]:
                landmark = results.face_landmarks.landmark[idx]
                x, y = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # Extract Right hand landmarks
            # right_hand = results.right_hand_landmarks.landmark
            # right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
            
            # Extract Left hand landmarks
            left_hand = results.left_hand_landmarks.landmark
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten()) if results.left_hand_landmarks else np.zeros(21*3)

            # Concate rows
            # row = pose_row+face_row+right_hand_row+left_hand_row
            row = pose_row+face_row+left_hand_row
            
            # Append class name 
            row.insert(0, class_name)
            # print(camera_id)
            #row.append(camera_id)
            num_frames = 30

           
            
        
            
            for i in range(num_frames):
                # Create a folder for the class if it doesn't exist for Camera 
                # keypoints = extract_keypoints(results)
                # npy_path = os.path.join(DATA_PATH, class_name, str(i))
                # np.save(npy_path, keypoints)
                file_path = os.path.join(DATA_PATH, class_name, f"camera_screenshot_{i + 1}.png")
            
                cv2.imwrite(file_path, image)
           
       

                file_name = f'coordsV2_{camera_id}_1.csv'
                file_path = os.path.join('/home/Downloads/Non_San/', file_name)
                with open(file_path, mode='w', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(landmarks)

            # Display the countdown on the screen
            countdown_text = f"Countdown: {int(remaining_time)} seconds"
            cv2.putText(image, countdown_text, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        
            
            # with open('coords_test0.csv', mode='a', newline='') as f:
            #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row) 

            # with open('coords_test1.csv', mode='a', newline='') as f:
            #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row) 
                # Export to CSV for Camera 1
            # Full path to the CSV file
            
            # with open(os.path.join(class_folder, f'coords_{camera_id}.csv'), mode='a', newline='') as f1:
            #     csv_writer1 = csv.writer(f1, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer1.writerow(row)
            
             
            
        except:
            pass



        cv2.imshow('Raw Webcam Feed', image)
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

pipeline.stop()
cv2.destroyAllWindows()

819312070630
0
819312070630


I0000 00:00:1741851332.570291    5763 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741851332.572996    5839 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: NV134
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
#Original
# Directory for saving images and data
# model_folder = 'Model'
# os.makedirs(model_folder, exist_ok=True)  # Create the "Model" folder if it doesn't exist
class_name = input("Enter the class name: ")


try: 
    
    os.makedirs(os.path.join(DATA_PATH, class_name), exist_ok=True)
    
except:
            pass

font = cv2.FONT_HERSHEY_COMPLEX
org = (20,100)
fontScale = .5
thickness = 1 
color = (0,150,255)
realsense_ctx = rs.context()
connected_devices = [] # List of serial numbers for present cameras
for i in range(len(realsense_ctx.devices)):
        detected_camera =  realsense_ctx.devices[i].get_info(rs.camera_info.serial_number)
        print(f"{detected_camera}")
        connected_devices.append(detected_camera)
camera_input = input("Select camera: ")
if camera_input == '0':
        camera_id = connected_devices[0]
elif camera_input == '1':
        camera_id = connected_devices[1]
print(camera_input)    
device = camera_id # In this example we are only using one camera  


print(camera_id)
# print(camera_id)



pipeline = rs.pipeline()    
config = rs.config()    
background_removed_color = 153 # Grey

config.enable_device(device)
stream_res_x = 640
stream_res_y = 480
stream_fps = 30
config.enable_stream(rs.stream.depth, stream_res_x, stream_res_y, rs.format.z16, stream_fps)
config.enable_stream(rs.stream.color, stream_res_x, stream_res_y, rs.format.bgr8, stream_fps)
profile = pipeline.start(config)
align_to = rs.stream.color
align = rs.align(align_to)
# ====== Get depth Scale ======
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
# ====== Set clipping distance ======
clipping_distance_in_meters = 2
clipping_distance = clipping_distance_in_meters / depth_scale

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    start_time = time.time()  
    while True:
        
        # start_time = dt.datetime.today().timestamp() # Necessary for FPS calculations

        # count down
        current_time = time.time()
        elapsed_time = current_time - start_time
        remaining_time = max(0, 30 - elapsed_time)  # Countdown for 30 seconds

        if remaining_time == 0:
            break

                # Get and align frames
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        if not aligned_depth_frame or not color_frame:
            continue

            # Process images
        # Wait for the next frame from the RealSense camera
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        
        if not depth_frame or not color_frame:
            continue

        # Convert the depth frame to a numpy array
        depth_image = np.asanyarray(depth_frame.get_data())

        # Convert the color frame to a numpy array
        color_image = np.asanyarray(color_frame.get_data())
        color_image = cv2.flip(color_image, 1) 
        
        # Apply colormap on depth image (for visualization)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            
        image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
                       # Image is no longer writeable
        
        # Recolor Feed
               
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten() if results.pose_landmarks else np.zeros(33*4) )
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten() if results.face_landmarks else np.zeros(468*3))

            # Extract Right hand landmarks
            right_hand = results.right_hand_landmarks.landmark
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten() if results.right_hand_landmarks else np.zeros(21*3))
            
            # Extract Left hand landmarks
            left_hand = results.left_hand_landmarks.landmark
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten()) if results.left_hand_landmarks else np.zeros(21*3)

            # Concate rows
            row = pose_row+face_row+right_hand_row+left_hand_row
            # row = pose_row+face_row+left_hand_row
            
            # Append class name 
            row.insert(0, class_name)
            # print(camera_id)
            #row.append(camera_id)
            num_frames = 30

           
            
        
            
            
            # Create a folder for the class if it doesn't exist for Camera 
            # keypoints = extract_keypoints(results)
            # npy_path = os.path.join(DATA_PATH, class_name, str(i))
            # np.save(npy_path, keypoints)
            file_path = os.path.join(DATA_PATH, class_name, f"camera_screenshot_{i + 1}.png")
        
            cv2.imwrite(file_path, image)
        
    

            file_name = f'coordsV2_{camera_id}.csv'
            file_path = os.path.join('/home/non/catkin_ws/src/robot_control/scripts/', file_name)
            with open(file_path, mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

            # Display the countdown on the screen
            countdown_text = f"Countdown: {int(remaining_time)} seconds"
            cv2.putText(image, countdown_text, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        
            
            # with open('coords_test0.csv', mode='a', newline='') as f:
            #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row) 

            # with open('coords_test1.csv', mode='a', newline='') as f:
            #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row) 
                # Export to CSV for Camera 1
            # Full path to the CSV file
            
            # with open(os.path.join(class_folder, f'coords_{camera_id}.csv'), mode='a', newline='') as f1:
            #     csv_writer1 = csv.writer(f1, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer1.writerow(row)
            
             
            
        except:
            pass



        cv2.imshow('Raw Webcam Feed', image)
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

pipeline.stop()
cv2.destroyAllWindows()

819312070630


IndexError: list index out of range

Normalization

Clustering

In [ ]:
data=[]
if results.pose_landmarks: 
    pose_data=[lm.x for lm in results.pose_landmarks.landmark] + \
              [lm.y for lm in results.pose_landmarks.landmark] + \
              [lm.z for lm in results.pose_landmarks.landmark]
    data.append(pose_data)

data=np.array(data)

num_clusters=5
kmeans=KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(data)

cluster_labels=kmeans.labels_

Read collect data CSV FILE

In [ ]:
df = pd.read_csv('coords.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Set up logging
logging.basicConfig(filename='trainingV3.log', level=logging.INFO)

In [ ]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
    'svm': make_pipeline(StandardScaler(), SVC()),
    'knn': make_pipeline(StandardScaler(), KNeighborsClassifier())
}

Train data with pipeline 

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    #model = LogisticRegression(solver='lbfgs', max_iter=30000000000000000)
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
fit_models

In [ ]:
fit_models['rc'].predict(X_test)

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
fit_models['rf'].predict(X_test)

SAVE

In [ ]:
with open('body_language_rf_15_03_24.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

: 

In [ ]:
with open('body_language_14_03_24.pkl', 'wb') as f:
    pickle.dump(model['rf'], f)

LOAD

In [ ]:
with open('body_language_normal__voting_14_03_24.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
fit_models['rf'].predict(X_test)

In [ ]:
model = pickle.load(open('body_language.pkl','rb'))

In [ ]:
fit_models['rf'].predict(X_test)

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.svm import SVC

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Get unique class labels from the true labels
class_labels = list(set(y_test))

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualize the confusion matrix
fig, ax = plt.subplots(figsize=(12, 10))  # Increase figure size for better readability
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=ax, annot_kws={"size": 10})  # Adjust annotation font size

# Improve the visualization
plt.xticks(np.arange(len(model.classes_)) + 0.5, model.classes_, rotation=45, ha='right')  # Adjust x-axis labels
plt.yticks(np.arange(len(model.classes_)) + 0.5, model.classes_, rotation=0, va='center')  # Adjust y-axis labels
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.tight_layout()  # Adjust layout to not cut off edge labels
plt.show()

In [ ]:
np.diag(conf_matrix ).sum()

In [ ]:
np.diag(conf_matrix ).sum()/conf_matrix .sum().sum()

In [ ]:
TP = conf_matrix [0, 0]
FP = conf_matrix [0, :].sum() - TP
FN = conf_matrix [:, 0].sum() - TP
TN = conf_matrix .sum() - (TP + FP + FN)


In [ ]:
# Print the values
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"True Negatives (TN): {TN}")

In [ ]:
# Calculate Accuracy
accuracy = (TP + TN) / conf_matrix .sum()

# Calculate Precision
precision = TP / (TP + FP)

# Calculate Recall
recall = TP / (TP + FN)

# Calculate F1 Score
f1_score = (2 * precision * recall) / (precision + recall)

# Print the values
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

In [ ]:
# Assuming your model is a scikit-learn model with the classes_ attribute
class_names = model.classes_

for i in range(conf_matrix .shape[0]):
    TP = conf_matrix [i, i]
    FP = conf_matrix [i, :].sum() - TP
    FN = conf_matrix [:, i].sum() - TP
    TN = conf_matrix .sum() - (TP + FP + FN)
    
    # Calculate Accuracy, Precision, Recall, and F1 Score
    accuracy = (TP + TN) / conf_matrix .sum()
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    
    # Print the results with real class names
    print(f"Class {class_names[i]} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")


Real-time Prediction

In [ ]:
threshold = 0.8
font = cv2.FONT_HERSHEY_COMPLEX
org = (20,100)
fontScale = .5
thickness = 1 
color = (0,150,255)
realsense_ctx = rs.context()
connected_devices = [] # List of serial numbers for present cameras
for i in range(len(realsense_ctx.devices)):
        detected_camera =  realsense_ctx.devices[i].get_info(rs.camera_info.serial_number)
        print(f"{detected_camera}")
        connected_devices.append(detected_camera)
        
device = connected_devices[0] # In this example we are only using one camera    
pipeline = rs.pipeline()    
config = rs.config()    
background_removed_color = 153 # Grey

config.enable_device(device)
stream_res_x = 640
stream_res_y = 480
stream_fps = 30
config.enable_stream(rs.stream.depth, stream_res_x, stream_res_y, rs.format.z16, stream_fps)
config.enable_stream(rs.stream.color, stream_res_x, stream_res_y, rs.format.bgr8, stream_fps)
profile = pipeline.start(config)
align_to = rs.stream.color
align = rs.align(align_to)
# ====== Get depth Scale ======
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
# ====== Set clipping distance ======
clipping_distance_in_meters = 2
clipping_distance = clipping_distance_in_meters / depth_scale

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while True:
        start_time = dt.datetime.today().timestamp() # Necessary for FPS calculations

                # Get and align frames
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        if not aligned_depth_frame or not color_frame:
            continue

            # Process images
        # Wait for the next frame from the RealSense camera
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert the depth frame to a numpy array
        depth_image = np.asanyarray(depth_frame.get_data())

        # Convert the color frame to a numpy array
        color_image = np.asanyarray(color_frame.get_data())
        color_image = cv2.flip(color_image, 1) 
        
        # Apply colormap on depth image (for visualization)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            
        image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
                       # Image is no longer writeable
        
        # Recolor Feed
               
               
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # Extract Right hand landmarks
            right_hand = results.right_hand_landmarks.landmark
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
            
             # Extract Left hand landmarks
            left_hand = results.left_hand_landmarks.landmark
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())

            # Concate rows
            row = pose_row+face_row+right_hand_row+left_hand_row
            # row = pose_row+face_row+left_hand_row
            
            
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            max_body_language_prob = round(body_language_prob[np.argmax(body_language_prob)])
            
            if max_body_language_prob > threshold:
                # Grab ear coords
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [640,480]).astype(int))
                

                cv2.rectangle(image, 
                            (coords[0], coords[1]+5), 
                            (coords[0]+len(body_language_class)*20, coords[1]-30), 
                            (245, 117, 16), -1)
                cv2.putText(image, body_language_class, coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
                
                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class.split(' ')[0]
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

pipeline.stop()
cv2.destroyAllWindows()